##### Importamos las libreria con que vamos a eleaborar la funciones

In [2]:
import numpy as np
import pandas as pd
import math
import ast

##### Importamos nuestra data

In [48]:
data_games=pd.read_csv('games.csv')
data_review=pd.read_csv('reviews.csv')
data_items=pd.read_csv('items.csv')

## Función numero uno

Vamos a crear una funcion que dado el id de un usuario nos devuelva la cantidad de dinero gastodo por este y el porcentaje de recomendacion

Miramos los valores unicos de la columna precio y se observa que hay valores de precio que no estan en numero

In [ ]:
data_games['price'].unique()

Del paso anterior extreaelos la valores que vamos a remplazar con numeros y creamos un diccionario de precios a remplazar

In [ ]:
remplazos_precio={'Free To Play': 0,
                  'Free to Play':0,
                   'nan':0,
                  'Free':0,
                  'Free Demo':0,
                  'Play for Free!':0,
                  'Install Now':0,
                  'Play WARMACHINE: Tactics Demo':0,
                  'Free Mod':0,
                  'Install Theme':0,
                  'Third-party':0,
                  'Play Now':0,
                  'Free HITMAN™ Holiday Pack':0,
                  'Play the Demo':0,
                  'Starting at $499.00':499,
                  'Starting at $449.00':499,
                  'Free to Try':0,
                  'Free Movie':0,
                  'Free to Use':0
                                                 
}

Realizamos el remplazo de los precio por los valores que se encuentra en el diccionario remplazos_precios

In [ ]:
data_games['price']=data_games['price'].replace(remplazos_precio)

Exportamos los cambios realizados al archvio data_games con el fin de no tener que realizar los pasos anteriores a la hora de correr nuestras funciones

data_games.to_csv('games.csv', index=False)

### Codigo de la funcion numero uno

In [87]:
# Ejemplo función
userdata('76561197970982479')

La cantidad de dinero gastado por es usuario es de: 3419.3199999999856 y el procentajes de recomendacion es: 1.3043478260869565


In [81]:
def userdata( User_id : str ):
    
    # creamos una maskara con la columna user_id y la guardamos en una lista (tolist())
    lista_juego=data_items[data_items['user_id']==User_id]['item_id'].tolist()
    
    # creamos una lista vacia
    lista_precio=[]

    # agregamos a nuestra lista aquellos precios que coinciden con id
    for i in lista_juego:

        lista_precio.append(data_games[data_games['id']==i]["price"].tolist())
        
    
    # contador de suma
    suma = 0  

    # este ciclo se encarga de sumar los valores de la lista y descartando aquellos valores nulos o vacios
    for i in lista_juego:
        prices = data_games[data_games['id'] == i]["price"].tolist()
        
        # Verifica si la lista de precios no está vacía
        if prices:
            # Filtra los valores no numéricos y nan y luego suma los valores
            validar_prices = [p for p in prices if isinstance(p, (int, float)) and not math.isnan(p)]
            # Suma los valores válidos de la lista de precios
            suma += sum(validar_prices)  
            
            
    cantidad_items=data_items[data_items['user_id']==User_id]['item_id'].count()
    cantidad_review=data_review[data_review['user_id']==User_id]['sentiment_analysis'].count()      
    recomendacion= (cantidad_review/cantidad_items)*100

    return(print('La cantidad de dinero gastado por es usuario es de:',suma, 'y el procentajes de recomendacion es:', recomendacion))


## Funcion numero dos

Esta funcion debe devolver la cantidad de usuariso que reviews realizaron reviews entre dos fechas dadas 

Para realizar esta funcions primero debemos de realizar un proceso de ETL para que estas puedan funcionar correctamente.
- Vamos a trabajar con la columna posted la cual contiene las fechas de las reviews. Pero antes debemos elimniar la palabra posted de nuestra culumna para quedarnos solo con las fechas

In [15]:
data_review['posted']=data_review['posted'].str.replace('Posted','')

In [16]:
data_review.head(2)

,user_id,user_url,posted,item_id,recommend_False,recommend_True,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"November 5, 2011.",1250,0,1,2.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"July 15, 2011.",22200,0,1,2.0


Eliminamos los puntos al final de cada una de nuestras fechas

In [88]:
data_review['posted']=data_review['posted'].str.replace('.','')

AttributeError: Can only use .str accessor with string values!

Como tenemos en nuestra columna posted tenemos fechas que no tiene año. Entonces procedemos a dividir a esta columna en dos y esta divicios se va hacer por la coma. Debido a que el año se encuestra antes de la coma. como resultados tendremos dos culumnas nuevas una de año de nombre year y otras de mes de nombre month

In [ ]:
data_review[['mont', 'year']]=data_review['posted'].str.split(',', expand=True)

Eliminamos el punto que se encuentra en nuestras columna year (año)

In [ ]:
data_review['year']=data_review['year'].str.replace('.','')

Filtramos todos los valores en donde nuestra columna year no es nula. Este paso lo realiazamos con el fin de que en nuestra columna posted no queden las fechas que tienen los daots completos como lo son mes, dia y año

In [ ]:
data_review=data_review[data_review['year'].notna()]

Cambiamos el formato de fecha de nuestra columna posted con el fin de que quede en el formato YY--MM--DD

In [ ]:
data_review['posted'] = pd.to_datetime(data_review['posted'], format='%B %d, %Y').dt.strftime('%Y-%m-%d')

#### Exportamos los cambios anteriores a nuestro archivo cvs de reviews.csv

Lo anterior lo hacemos con el fin de no tener quener que hacer ningun proceso de ETL para poder correr nuestras funciones

In [84]:
data_review.to_csv('reviews.csv', index=False)

## Codigo función numero dos

In [86]:
# Ingresar la fechas en formato YYYY-MM-DD
countreviews('2011-01-01', '2014-12-31' )

16605


In [85]:
def countreviews( fecha_inicio,  fecha_fin : str ):
    ## nos aseguramos que le columna posted tenga el formato adecuado
    data_review['posted'] = pd.to_datetime(data_review['posted'], format='%Y-%m-%d')

    ## nos aseguramos que las fechas ingresadas  tenga el formato adecuado
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)
    return(print(len(data_review[(data_review['posted']>= fecha_inicio) & (data_review['posted']<=fecha_fin)]['user_id'].unique())))

## Función numero tres

Antes de inicar a eleaborar la funcion numeros tres debemos de realizar un proceso de ETL de nuestro dataFRame (datya_games). el primer paso que vamos a realizar es de filtrar aquella columna donde genres y tags son vacias, con el fin de descartar aquellos viedo juegos que no tenga genero

In [204]:
data_games = data_games[(data_games['genres'].notna()) & (data_games['tags'].notna())]

Como nuestra columna genres esta conpuesto por filas llenas de lista las cuales tienen formato tipo strin debemos de convertir esta columna de tipo strin a tipo lista

In [ ]:
data_games['genres'] = data_games['genres'].apply(ast.literal_eval)

Obtenemos el primer elemento de las lista que se encuestran dentro de la columna genres y la guardamos en una nueva columna de nobre primer_genero

In [ ]:
data_games['primer_genero'] = data_games['genres'].str.get(0)

Observamos una lista de unicos elemento de la columna genres y podemos detectar que exiten generos que no pretenece a video juegos

In [ ]:
data_games['primer_genero'].unique()

array(['Action', 'Free to Play', 'Casual', 'Adventure', 'Racing',
       'Strategy', 'Indie', 'RPG', 'Simulation', 'Sports',
       'Massively Multiplayer', 'Animation &amp; Modeling', 'Utilities',
       'Education', 'Design &amp; Illustration', 'Audio Production',
       'Video Production', 'Software Training', 'Accounting',
       'Web Publishing', 'Photo Editing', 'Early Access'], dtype=object) 

En base al anterior paso eleaboramos una lista de los generos que no pertenecen a la categoria de video juego con el fin de desvartarlos

In [ ]:
genero_descartar= ['Animation &amp; Modeling', 'Utilities',
       'Education', 'Design &amp; Illustration', 'Audio Production',
       'Video Production', 'Software Training', 'Accounting',
       'Web Publishing', 'Photo Editing', 'Early Access']

Filtramos aquellos juegos que no pertencen a la lista genero_descartar

In [ ]:
data_games['genres_games']=data_games[~data_games['primer_genero'].isin(genero_descartar)]['primer_genero'] # ~ se usa para negar la condicion

De nuestro dataFrame genres_games vamos a crear un dataFrame auxiliar que contengas solo las columnas id y genres_games con el proposito de realizar un iner join con el DataFrame data_items

In [ ]:
df_auxiliar= data_games[['genres_games', 'id']]

Cambiamos el nombre de la columna id a item_id para que concida con el nombre de la columna que tenemos en el archivo data_items y poder hacer el iner join

In [ ]:
## cambiamos el nombre de la columna id a item_id para que concida con la columna que tenemos en el archivo data_items
df_auxiliar['item_id']=df_auxiliar['id']

## eiminamos la columna id
df_auxiliar=df_auxiliar.drop(columns='id')

Unimos los dos dataFrame a traves de la columna item_id

In [ ]:
data_items=pd.merge(data_items, df_auxiliar, on='item_id', how='inner')

Filtramos los valores donde la columa generes_games es diferentes a vacios

In [ ]:
data_items=data_items[data_items['genres_games'].notna()]

## Nota:

El dataFRame original de data_itmes originalmente tenia 5153209 fila y cundo hacemos el filtro esta psa a tenes 4188785 filas debido a que como eliminamos categorias que no pertenecia a video juegos y por ende eliminamos item_id que no correspondian a niguna categoria de video juego por lo cual el numero de fila se vio reducida

In [ ]:
data_items.shape

#### Exportamos estos cambios a con el fin de no tener que volver hecer todo este proceso de ETL para poder correr nuestras funciones directamente

In [ ]:
data_items.to_csv('items.csv', index=False)

## Codigo de la función numero tres

In [64]:
# INGRESA UN GENERO 
genre('Action')

,user_id,user_url,playtime_forever
0,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699287
1,shinomegami,http://steamcommunity.com/id/shinomegami,1580428
2,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212
3,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742
4,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061086


In [63]:
def genre(genero:str):
    # usamos una maskara para filtrar las columnas con que vamos a trabajar
    ranking = data_items[data_items['genres_games'] == genero][['user_id', 'user_url', 'playtime_forever']]

    # Realizar la operación de agrupación y suma
    ranking = ranking.groupby('user_id').agg({'playtime_forever': 'sum', 'user_url': 'first'})

    # Ordenar el ranking por 'playtime_forever'
    ranking = ranking.sort_values(by='playtime_forever', ascending=False).reset_index()

    # ordenamos las columnas
    ranking=ranking[['user_id','user_url','playtime_forever']]

    # Mostrar las 5 mejores filas
    ranking.head(5)
    
    return(ranking.head(5))
